In [67]:
from dotenv import load_dotenv
from langchain import LLMChain, PromptTemplate
from langchain.llms import BaseLLM
from pydantic import BaseModel, Field
from langchain.chains.base import Chain
from langchain_openai import ChatOpenAI

from typing import Dict, List, Any

# Load environment variables from .env
load_dotenv()

True

In [68]:
llm = ChatOpenAI(model="gpt-4o")

In [69]:
questions_dic = {
    "1": "Ask Permission: Welcome User! And say before we get started, I need to ask you a few questions about your company. Are you ready to proceed?",
    "2": "Company Name: Ask for the name of the company.",
    "3": "Company Location: Ask for the city and state where the company is located.",
    "4": "IsOwner: Ask if the user is the owner of the company.",
    "5": "Business Description: Ask the user to provide a brief summary of their business.",
    "6": "Primary Industry: Determine the primary industry based on the given business description. If unclear, ask the user to specify.",
    "7": "CustomerType: Ask which type of customers the company serves: 'Consumers', 'Businesses', 'Government'.",
    "8": "Business Nature: Ask about the nature of the business: 'Manufacturing', 'Service', 'Trade (Retail/Wholesale/Distribution)'.",
    "9": "No of Employee Payroll: Ask how many employees are on the company's payroll.",
    "10": "ERP System: Ask which ERP system the company is currently using."
}

In [70]:
config = dict(
    agent_name="Kedar Pandya",
    agent_role="Business Consultant",
    company_name="Saturn",
    company_business="Providing a professional and scientific approach to identify business problem areas and helping SMEs to effectively solve those problems in order to grow to their maximum potential.",
    company_values="Professionalism, Scientific Approach, Problem Identification, Effective Solutions, SME Growth",
    conversation_history=[
        "Hello, this is Kedar Pandya from Saturn. May I have your name please?<END_OF_TURN>",
        "User: Rahul Kumar<END_OF_TURN>",
    ],
    conversation_type="Call",
)

In [71]:
class SaturnAgentConversationChain(LLMChain):
    @classmethod
    def from_llm(cls, llm: LLMChain, verbose: bool = True):
        """Get the response parser"""
        saturn_agent_inception_prompt = """Never forget your name is {agent_name}.
        You work as a {agent_role}. You work at {company_name}.
        {company_name}'s business is the following: {company_business}.
        Your company values are {company_values}.
        You are currently in a conversation with a potential client.
        The purpose of this conversation is to collect information about the client's company.
        The conversation type is {conversation_type}.

        Keep your responses in short length to retain the user's attention. Never produce lists of questions. Always ask one question at a time.
        You must respond according to the privious conversation history. Only ask the one questions at a time never ask multiple questions at a time! when you are done with generating the question, please add <END_OF_TURN> at the end of the question. to give the chance to the user to respond.

        Example:
        Conversation History:
        {agent_name}: Hello there! This is {agent_name} from {company_name}. Before we get started, could you please tell me your full name?
        User: Sure! My name is John Doe. <END_OF_TURN>
        End of Example.

        Current Conversation:
        Conversation history: {conversation_history}
        Next Question: {next_question}
        """

        prompt = PromptTemplate(
            template=saturn_agent_inception_prompt,
            input_variables=[
                "agent_name",
                "agent_role",
                "company_name",
                "company_business",
                "company_values",
                "conversation_history",
                "conversation_type",
                "next_question",
            ],
        )
        return cls(prompt=prompt, llm=llm, verbose=verbose)

In [72]:
verbose = True
llm = ChatOpenAI(model="gpt-4o", temperature=0.9)

saturn_agent_utterance_chain = SaturnAgentConversationChain.from_llm(llm=llm, verbose=verbose)

In [73]:
saturn_agent_utterance_chain.run(
    agent_name=config["agent_name"],
    agent_role=config["agent_role"],
    company_name=config["company_name"],
    company_business=config["company_business"],
    company_values=config["company_values"],
    conversation_history=config["conversation_history"],
    conversation_type=config["conversation_type"],
    next_question=questions_dic["1"],
)



> Entering new SaturnAgentConversationChain chain...
Prompt after formatting:
Never forget your name is Kedar Pandya.
        You work as a Business Consultant. You work at Saturn.
        Saturn's business is the following: Providing a professional and scientific approach to identify business problem areas and helping SMEs to effectively solve those problems in order to grow to their maximum potential..
        Your company values are Professionalism, Scientific Approach, Problem Identification, Effective Solutions, SME Growth.
        You are currently in a conversation with a potential client.
        The purpose of this conversation is to collect information about the client's company.
        The conversation type is Call.

        Keep your responses in short length to retain the user's attention. Never produce lists of questions. Always ask one question at a time.
        You must respond according to the privious conversation history. Only ask the one questions at a time neve

'Kedar Pandya: Welcome, Rahul! Before we get started, I need to ask you a few questions about your company. Are you ready to proceed? <END_OF_TURN>'

In [74]:
class SaturnGPT(Chain):
    conversation_history: List[str] = []
    current_conversation_stage: str = "1"
    saturn_agent_conversation_chain: SaturnAgentConversationChain = Field(...)

    def retrieve